In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [4]:
# read table

dataPayment = './Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [5]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

In [6]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('./cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('./cache'):
        os.mkdir('./cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id)
    with open('cache/ids.txt', 'r') as f:
        arr = []
        for _id in f:
            arr.append(int(_id)) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [69]:
emps = []
max = 0;
for _id in arr:
    df = payroll[payroll['Job Code'] == _id]
    emp_id = df['Employee Identifier'].unique()
    if len(emp_id) > 50:
        l = 75
    else:
        l = len(emp_id)

    for i in range(l):
        for i in range(100):
            _index = np.random.randint(0, l-1)
            if emp_id[_index] not in emps:
                emps.append(emp_id[_index])
                break


len(emps)

2705

In [70]:
df_id = payroll[payroll['Employee Identifier'].isin(emps)]
df_id

,Fiscal Year,Fiscal Period,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Working Year
0,2016,2016.00,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.00
1,2016,2016.25,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.25
2,2016,2016.50,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.50
3,2016,2016.75,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.75
4,2017,2017.00,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.00
...,...,...,...,...,...,...,...,...
234225,2018,2018.00,1570,Probation Officer II- PSB,13649.92,9514924,f01aa1fa-b791-4a57-b082-1de8559d5fba,34.00
234254,2018,2018.25,606,Assistant Public Defender III,30870.00,9511966,12c5ef30-094f-460d-98f6-3debea6853be,25.25
234259,2018,2018.00,48,Administrative Assistant III,17060.50,9513388,6ab6e3e3-8a44-4d22-8439-fee2ea9f9db6,25.00
234291,2018,2018.25,2051,Pharmacy Tech ARNTE,13964.56,903735,83c847da-1d2a-4a5c-b057-8cc02b2da114,4.25


In [71]:
X = df_id[['Job Code','Working Year','Position ID','Fiscal Period', 'Fiscal Year']]
Y = df_id['Base Pay']

In [65]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size =0.2)

In [56]:
from sklearn.neighbors import KNeighborsRegressor
# Create KNN classifier
knn = KNeighborsRegressor(n_neighbors = 3)
# Fit the classifier to the data
knn.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=3)

In [57]:
y_pred = knn.predict(x_test)

In [58]:
knn.score(x_test, y_test)

0.6713053453824529

In [59]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

2976.055022572178

In [72]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RidgeCV
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

RandomForestRegressor()

In [73]:
y_pred = rf.predict(x_test)

In [74]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)
# accuracy_score(y_test, y_pred)

1996.5955421311717